In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [62]:
import sys
sys.path.append('../src/')

import data.reader as dataReader
import utils.utils as thesisUtils
import similarities.cosine as thesisCosineSimilarity
import vocabulary.vocabulary as thesisVocabulary

import imp
imp.reload(thesisUtils)
imp.reload(dataReader)
imp.reload(thesisVocabulary)
imp.reload(thesisCosineSimilarity)

<module 'similarities.cosine' from '../src/similarities/cosine.py'>

In [63]:
SIMILARITY_SCORE_UPPER_THRESHOLD = 0.8

N_GRAM_RANGE_FOR_SHARED_VOCABULARY_FROM = 36
N_GRAM_RANGE_FOR_SHARED_VOCABULARY_TO = 29

In [64]:
london_corpus = dataReader.CorpusByNewLine.london()
zwickau_corpus = dataReader.CorpusByNewLine.zwickau()

In [ ]:
london_zwickau_similarities = thesisCosineSimilarity.CrossVersionSimilarity5Gram(london_corpus, zwickau_corpus)
london_zwickau_similarities.calculate()
london_zwickau_similarities.text_alignment_df()

In [45]:
zwickau_london_similarities = thesisCosineSimilarity.CrossVersionSimilarity5Gram(zwickau_corpus, london_corpus)
zwickau_london_similarities.calculate()
zwickau_london_similarities.text_alignment_df()

,zwickau,london,score
0,incipit descriptio terre sancte,hucusque de situ terrarum terre sancte adiacen...,0.170487
1,cum sicut dicit ieronimus quosdam inueniamus l...,cum in ueteribus historiis legamus sicut dicit...,0.811162
2,quis cristianus hiis uisis non festinabit ueni...,quis cristianus hiis uisis non festinet uenire...,0.895520
3,redeat post in ierusalem uideat et audiat in t...,redeat post hec in ierusalem ut uideat et audi...,0.888805
4,sunt preterea quedam que in locis ilis deuotio...,sunt preterea quedam que in locis ilis deuotio...,0.812920
...,...,...,...
316,de strongoli igitur mouentes auta et sperantes...,de strongoli igitur mouentes auta et sperantes...,0.917646
317,egressi itaque de galea uenimus baias de quo l...,egressi itaque de galea uenimus boias de quo l...,0.890851
318,in neapoli in castro saluatoris uidi ouum uirg...,in neapoli in castro saluatoris uidi ouum uirg...,0.856911
319,de roma per arecium et florentiam ueni bonnoni...,de roma per arecium et florentiam ueni bononia...,0.841701


In [ ]:
print(len(london_zwickau_similarities.get_matches_higher_than(SIMILARITY_SCORE_UPPER_THRESHOLD)))
print(len(zwickau_london_similarities.get_matches_higher_than(SIMILARITY_SCORE_UPPER_THRESHOLD)))

In [58]:
london_zwickau_similarities.get_matches_higher_than(SIMILARITY_SCORE_UPPER_THRESHOLD)

[1 -> 1: 0.8131045786315674,
 2 -> 2: 0.8958853131371243,
 3 -> 3: 0.8903220617363937,
 4 -> 4: 0.8165486916831831,
 5 -> 5: 0.8642997597862219,
 8 -> 8: 0.8293117327749867,
 40 -> 40: 0.8063132048187789,
 48 -> 51: 0.818124147457859,
 117 -> 122: 0.817103347087383,
 119 -> 124: 0.8077434386502398,
 166 -> 176: 0.844067389629243,
 169 -> 179: 0.8098589830818179,
 185 -> 194: 0.8002479175280295,
 192 -> 169: 0.8654566572032558,
 193 -> 170: 0.8623420097819804,
 237 -> 233: 0.8038463459700745,
 238 -> 234: 0.8884756958325792,
 239 -> 235: 0.8064803973318556,
 241 -> 237: 0.8170656962023194,
 243 -> 239: 0.8568922682825689,
 244 -> 240: 0.9605102439657496,
 245 -> 241: 0.9567323366506958,
 246 -> 242: 0.9837316304424514,
 247 -> 243: 0.8650658420886953,
 248 -> 244: 0.9262722996579451,
 249 -> 245: 0.9669399419309315,
 251 -> 247: 0.9710170154299992,
 252 -> 248: 0.9743336173709812,
 253 -> 249: 0.8862986006180578,
 254 -> 250: 0.84493799069109,
 260 -> 254: 0.9006783854967839,
 261 -> 26

In [31]:
london_zwickau_p_to_remove_by_shared_word = []
zwickau_london_p_to_remove_by_shared_word = []

for n_gram_range in range(N_GRAM_RANGE_FOR_SHARED_VOCABULARY_FROM, N_GRAM_RANGE_FOR_SHARED_VOCABULARY_TO, -1):
    shared_words = thesisVocabulary.get_2_version_shared_words(
        london_corpus.get_n_grams_words_dictionary(n_gram_range, n_gram_range), london_corpus.corpus_name,
        zwickau_corpus.get_n_grams_words_dictionary(n_gram_range, n_gram_range), zwickau_corpus.corpus_name,
    )
    for shared_word in shared_words:
        london_indexes_for_shared_word = thesisUtils.find_text_p_indexed_in_corpus(london_corpus.corpus, shared_word)
        zwickau_indexes_for_shared_word = thesisUtils.find_text_p_indexed_in_corpus(zwickau_corpus.corpus, shared_word)

        london_zwickau_p_to_remove_by_shared_word = london_zwickau_p_to_remove_by_shared_word + london_indexes_for_shared_word
        zwickau_london_p_to_remove_by_shared_word = zwickau_london_p_to_remove_by_shared_word + zwickau_indexes_for_shared_word

In [54]:
set(zwickau_london_p_to_remove_by_shared_word)

{5,
 226,
 240,
 241,
 247,
 254,
 271,
 272,
 273,
 279,
 280,
 281,
 283,
 286,
 288,
 293,
 296,
 298,
 299,
 300,
 302,
 307,
 310,
 312}

In [55]:
set(london_zwickau_p_to_remove_by_shared_word)

{5,
 232,
 244,
 245,
 251,
 260,
 263,
 264,
 265,
 271,
 272,
 273,
 275,
 278,
 280,
 285,
 288,
 290,
 291,
 292,
 294,
 299,
 302,
 304}

In [56]:
print(set([ i.original_index for i in london_zwickau_similarities.get_matches_higher_than(SIMILARITY_SCORE_UPPER_THRESHOLD) ]).union(set(london_zwickau_p_to_remove_by_shared_word)))

{1, 2, 3, 4, 5, 260, 261, 8, 262, 263, 264, 265, 266, 268, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 40, 295, 296, 297, 298, 299, 300, 301, 48, 302, 304, 305, 306, 307, 308, 309, 310, 311, 117, 119, 166, 169, 185, 192, 193, 232, 237, 238, 239, 241, 243, 244, 245, 246, 247, 248, 249, 251, 252, 253, 254}


In [52]:
print(set([ i.original_index for i in zwickau_london_similarities.get_matches_higher_than(SIMILARITY_SCORE_UPPER_THRESHOLD) ]).union(set(zwickau_london_p_to_remove_by_shared_word)))

{1, 2, 3, 4, 5, 8, 268, 269, 271, 272, 273, 274, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 40, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 51, 306, 307, 308, 309, 310, 312, 313, 314, 315, 316, 317, 318, 319, 122, 124, 169, 170, 176, 179, 226, 233, 234, 235, 237, 239, 240, 241, 242, 243, 244, 245, 247, 248, 249, 250, 254}
